In [4]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [5]:
df = pd.read_csv("./data/processed_retail_sales_dataset.csv")
df.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.30,0.016047,249.8092,1999,3735.1380,FDA15,Low Fat,Dairy,OUT049,Medium,Tier 1,Supermarket Type1
1,5.92,0.019278,48.2692,2009,443.4228,DRC01,Regular,Soft Drinks,OUT018,Medium,Tier 3,Supermarket Type2
2,17.50,0.016760,141.6180,1999,2097.2700,FDN15,Low Fat,Meat,OUT049,Medium,Tier 1,Supermarket Type1
3,19.20,0.000000,182.0950,1998,732.3800,FDX07,Regular,Fruits and Vegetables,OUT010,Medium,Tier 3,Grocery Store
4,8.93,0.000000,53.8614,1987,994.7052,NCD19,Low Fat,Household,OUT013,High,Tier 3,Supermarket Type1


In [21]:
X = df.drop(['Item_Outlet_Sales','Item_Identifier'],axis=1)

In [8]:
df.columns

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Item_Outlet_Sales', 'Item_Identifier',
       'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [12]:
# define numerical & categorical columns
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print("Numeric features: ",numeric_features)
print("Categorical features: ",categorical_features)

Numeric features:  ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales']
Categorical features:  ['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


In [24]:
y = df['Item_Outlet_Sales']

In [17]:
def showCategorical(df1:pd.DataFrame):
    categorical_features = [feature for feature in df1.columns if df1[feature].dtype == 'O']
    for column in categorical_features:
            print(f"Categories in {column} variable:",end="")
            print(df1[column].unique())
showCategorical(X)

Categories in Item_Fat_Content variable:['Low Fat' 'Regular']
Categories in Item_Type variable:['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']
Categories in Outlet_Identifier variable:['OUT049' 'OUT018' 'OUT010' 'OUT013' 'OUT027' 'OUT045' 'OUT017' 'OUT046'
 'OUT035' 'OUT019']
Categories in Outlet_Size variable:['Medium' 'High' 'Small']
Categories in Outlet_Location_Type variable:['Tier 1' 'Tier 3' 'Tier 2']
Categories in Outlet_Type variable:['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']


In [19]:
X.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.30,0.016047,249.8092,1999,Low Fat,Dairy,OUT049,Medium,Tier 1,Supermarket Type1
1,5.92,0.019278,48.2692,2009,Regular,Soft Drinks,OUT018,Medium,Tier 3,Supermarket Type2
2,17.50,0.016760,141.6180,1999,Low Fat,Meat,OUT049,Medium,Tier 1,Supermarket Type1
3,19.20,0.000000,182.0950,1998,Regular,Fruits and Vegetables,OUT010,Medium,Tier 3,Grocery Store
4,8.93,0.000000,53.8614,1987,Low Fat,Household,OUT013,High,Tier 3,Supermarket Type1


In [22]:
Y.head()

0    3735.1380
1     443.4228
2    2097.2700
3     732.3800
4     994.7052
Name: Item_Outlet_Sales, dtype: float64

In [23]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)
X = preprocessor.fit_transform(X)
X.shape #after OHE, x has 42 columns and 8523 rows

(8523, 42)

In [25]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((6818, 42), (1705, 42))

In [26]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [27]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 1141.5316
- Mean Absolute Error: 847.2209
- R2 Score: 0.5595
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1069.3653
- Mean Absolute Error: 792.0250
- R2 Score: 0.5793


Lasso
Model performance for Training set
- Root Mean Squared Error: 1141.7272
- Mean Absolute Error: 846.8713
- R2 Score: 0.5593
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1068.5936
- Mean Absolute Error: 791.2851
- R2 Score: 0.5799


Ridge
Model performance for Training set
- Root Mean Squared Error: 1141.5318
- Mean Absolute Error: 847.1956
- R2 Score: 0.5595
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1069.3552
- Mean Absolute Error: 792.0093
- R2 Score: 0.5793


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 997.3556
- Mean Absolute Error: 716.1162
- R2 Sco

In [28]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
7,CatBoosting Regressor,0.589194
1,Lasso,0.579873
2,Ridge,0.579274
0,Linear Regression,0.579266
5,Random Forest Regressor,0.566885
6,XGBRegressor,0.528976
8,AdaBoost Regressor,0.528134
3,K-Neighbors Regressor,0.504428
4,Decision Tree,0.174255


In [29]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 57.93


In [30]:
rfr_model = RandomForestRegressor()
rfr_model = rfr_model.fit(X_train, y_train)
y_pred = rfr_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 56.32


In [31]:
cbr_model = CatBoostRegressor(
    iterations=1000,           # Number of trees
    learning_rate=0.1,        # Step size
    depth=6,                  # Depth of trees
    l2_leaf_reg=3,            # L2 regularization
    eval_metric='RMSE',       # Evaluation metric
    random_seed=42            # For reproducibility
)
cbr_model = rfr_model.fit(X_train, y_train)
y_pred = cbr_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 56.52
